In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,0,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
0,53.27,0.00,-0.00,-0.00,0.00,0.00,-0.0,-0.00,0.00,-0.00,...,-0.00,-0.00,-0.00,-0.0,0.00,-0.00,0.00,0.0,0.00,0.00
2,0.00,72.91,-0.00,-0.00,-0.00,-0.00,0.0,0.00,-0.00,-0.00,...,-0.00,0.00,-0.00,-0.0,0.00,-0.00,0.00,-0.0,-0.00,0.00
3,-0.00,-0.00,21.12,-0.00,-0.00,0.00,0.0,0.00,0.00,0.00,...,-0.00,0.00,0.00,0.0,-0.00,0.00,0.00,0.0,-0.00,0.00
4,-0.00,-0.00,-0.00,87.97,0.00,0.00,-0.0,0.00,-0.00,-0.00,...,0.00,0.00,0.00,0.0,0.00,-0.00,0.00,-0.0,0.00,-0.00
5,0.00,-0.00,-0.00,0.00,55.22,-0.00,-0.0,0.00,-0.00,-0.00,...,0.00,0.00,0.00,-0.0,-0.00,-0.00,0.00,-0.0,0.00,0.00
6,0.00,-0.00,0.00,0.00,-0.00,40.17,0.0,-0.00,0.00,0.00,...,-0.00,0.00,0.00,-0.0,0.00,-0.00,0.00,-0.0,-0.00,0.00
7,-0.00,0.00,0.00,-0.00,-0.00,0.00,22.8,0.00,0.00,0.00,...,-0.00,0.00,-0.00,-0.0,0.00,-0.00,0.00,0.0,0.00,-0.00
8,-0.00,0.00,0.00,0.00,0.00,-0.00,0.0,1.78,0.00,-0.00,...,-0.00,0.00,-0.00,-0.0,-0.00,-0.00,0.00,0.0,0.00,0.00
9,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.0,0.00,8.49,0.00,...,-0.00,0.00,0.00,-0.0,-0.00,-0.00,-0.00,0.0,-0.00,-0.00
10,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.0,-0.00,0.00,33.38,...,0.00,-0.00,0.00,-0.0,0.00,0.00,-0.00,0.0,-0.00,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,0,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
0,1.00000,0.00002,-0.00000,-0.00003,0.00002,0.00006,-0.00002,-0.00008,0.00002,-0.00001,...,-0.00001,-0.00003,-0.00006,-0.00001,0.00001,-0.00001,0.00001,0.00000,0.00004,0.00004
2,0.00002,1.00000,-0.00002,-0.00006,-0.00002,-0.00006,0.00001,0.00003,-0.00000,-0.00005,...,-0.00000,0.00000,-0.00004,-0.00005,0.00000,-0.00004,0.00001,-0.00001,-0.00003,0.00003
3,-0.00000,-0.00002,1.00000,-0.00001,-0.00000,0.00003,0.00002,0.00007,0.00002,0.00004,...,-0.00003,0.00005,0.00000,0.00002,-0.00002,0.00002,0.00004,0.00004,-0.00001,0.00003
4,-0.00003,-0.00006,-0.00001,1.00000,0.00003,0.00002,-0.00002,0.00006,-0.00001,-0.00001,...,0.00006,0.00001,0.00003,0.00003,0.00001,-0.00003,0.00001,-0.00003,0.00003,-0.00005
5,0.00002,-0.00002,-0.00000,0.00003,1.00000,-0.00005,-0.00001,0.00002,-0.00002,-0.00003,...,0.00002,0.00002,0.00001,-0.00001,-0.00001,-0.00004,0.00000,-0.00001,0.00005,0.00003
6,0.00006,-0.00006,0.00003,0.00002,-0.00005,1.00000,0.00000,-0.00001,0.00003,0.00001,...,-0.00000,0.00002,0.00001,-0.00001,0.00006,-0.00001,0.00001,-0.00002,-0.00001,0.00006
7,-0.00002,0.00001,0.00002,-0.00002,-0.00001,0.00000,1.00000,0.00001,0.00001,0.00000,...,-0.00006,0.00003,-0.00002,-0.00003,0.00001,-0.00002,0.00006,0.00001,0.00001,-0.00003
8,-0.00008,0.00003,0.00007,0.00006,0.00002,-0.00001,0.00001,1.00000,0.00002,-0.00001,...,-0.00001,0.00005,-0.00001,-0.00004,-0.00002,-0.00002,0.00001,0.00005,0.00001,0.00000
9,0.00002,-0.00000,0.00002,-0.00001,-0.00002,0.00003,0.00001,0.00002,1.00000,0.00001,...,-0.00000,0.00004,0.00003,-0.00006,-0.00001,-0.00005,-0.00006,0.00000,-0.00005,-0.00002
10,-0.00001,-0.00005,0.00004,-0.00001,-0.00003,0.00001,0.00000,-0.00001,0.00001,1.00000,...,0.00000,-0.00001,0.00003,-0.00001,0.00003,0.00003,-0.00001,0.00001,-0.00000,0.00001


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.06162433280011647

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.38901395e+02 9.67793522e+01 9.54107277e+01 9.20922388e+01
 8.65757340e+01 8.22135399e+01 6.85261174e+01 6.15045808e+01
 6.04581091e+01 5.81929148e+01 5.47938986e+01 5.22522708e+01
 5.14214894e+01 4.88678385e+01 4.12873082e+01 3.97157697e+01
 3.83070451e+01 3.54753808e+01 3.43079668e+01 3.19984842e+01
 3.03262286e+01 2.96902156e+01 2.89549856e+01 2.82332638e+01
 2.62666402e+01 2.35897646e+01 2.26340658e+01 2.14504416e+01
 2.03069532e+01 1.86686025e+01 1.54789429e+01 1.45742297e+01
 1.41050489e+01 1.33009708e+01 1.31398345e+01 1.26591865e+01
 1.14897077e+01 1.03849289e+01 9.70185516e+00 8.82989433e+00
 7.86940172e+00 7.27678168e+00 6.71577559e+00 6.17147699e+00
 5.91437388e+00 5.84540202e+00 5.30077313e+00 4.66126938e+00
 3.87658724e+00 2.23012077e+00 1.34854466e+00 1.27794652e+00
 8.08282420e-01 5.03061857e-01 1.44054142e-01 1.75956613e-30]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,-0.012590,-0.035733,-0.003321,0.996026,-0.013514,-0.007497,-0.003628,-0.000254,-0.001271,-0.005782,...,-0.000858,-0.004656,-0.018133,-0.009487,-0.003165,-0.032526,-0.000714,-0.002897,-0.015268,-0.002183
1,-0.001419,-0.654632,-0.000261,0.002152,-0.001328,-0.000615,-0.000354,-0.000031,-0.000105,-0.000482,...,-0.000084,-0.000374,-0.002402,-0.000951,-0.000257,-0.022192,-0.000058,-0.000201,-0.001881,-0.000206
2,-0.014094,0.679411,-0.002821,0.022786,-0.015918,-0.007136,-0.003106,-0.000217,-0.001043,-0.005234,...,-0.000733,-0.004039,-0.027637,-0.009717,-0.002663,-0.435500,-0.000584,-0.002407,-0.020002,-0.001813
3,-0.022437,0.274060,-0.004275,0.031107,-0.025743,-0.010986,-0.004723,-0.000333,-0.001616,-0.007916,...,-0.001110,-0.006185,-0.046159,-0.015042,-0.004102,0.869001,-0.000898,-0.003674,-0.032034,-0.002718
4,-0.021371,0.101796,-0.003814,0.022792,-0.024870,-0.009861,-0.004162,-0.000284,-0.001399,-0.007150,...,-0.000955,-0.005524,-0.050479,-0.013680,-0.003633,0.147701,-0.000785,-0.003248,-0.032458,-0.002431
5,-0.039701,0.115398,-0.006477,0.034327,-0.047146,-0.017252,-0.007187,-0.000499,-0.002394,-0.012253,...,-0.001667,-0.009545,-0.109222,-0.024596,-0.006228,0.145321,-0.001369,-0.005608,-0.063242,-0.004175
6,-0.039883,0.032698,-0.004370,0.015211,-0.052784,-0.012828,-0.004859,-0.000328,-0.001569,-0.008629,...,-0.001097,-0.006543,0.976032,-0.019761,-0.004171,0.035917,-0.000901,-0.003772,-0.099785,-0.002757
7,-0.012576,0.004793,-0.000944,0.002590,-0.021435,-0.002816,-0.001046,-0.000067,-0.000318,-0.001822,...,-0.000261,-0.001372,0.019035,-0.004836,-0.000853,0.005233,-0.000190,-0.000744,-0.587908,-0.000586
8,-0.060981,0.020887,-0.004149,0.011105,-0.111064,-0.013187,-0.004599,-0.000301,-0.001492,-0.008481,...,-0.001023,-0.006243,0.071274,-0.022448,-0.003927,0.021993,-0.000817,-0.003565,0.728698,-0.002526
9,-0.112077,0.026113,-0.005866,0.014849,-0.276927,-0.019267,-0.006458,-0.000422,-0.002069,-0.012134,...,-0.001430,-0.008956,0.076701,-0.033716,-0.005549,0.028137,-0.001155,-0.004928,0.262471,-0.003625


In [13]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine')
cos_dist = np.sort(np.concatenate(cos_dist))[:len(GROUPS)]
cos_dist


array([0.00397416, 0.02396824, 0.06909046, 0.08197709, 0.08901983,
       0.0899352 , 0.0950722 , 0.09863487, 0.10897704, 0.12497131,
       0.130999  , 0.13110385, 0.14046203, 0.14730552, 0.14796754,
       0.1572291 , 0.16515318, 0.16898629, 0.1715707 , 0.18606714,
       0.19400272, 0.19689951, 0.19772206, 0.19915246, 0.20853206,
       0.21611841, 0.21889033, 0.22315791, 0.22914318, 0.23112495,
       0.23959286, 0.24180902, 0.24447669, 0.24714763, 0.26032796,
       0.26128733, 0.26739095, 0.27130213, 0.27132521, 0.28041703,
       0.28636549, 0.290798  , 0.29284293, 0.29314124, 0.30628979,
       0.31975458, 0.32058926, 0.33188229, 0.3525459 , 0.39667975,
       0.40824592, 0.40916825, 0.42636827, 0.45940789, 0.46062035,
       0.48109345])